Based on https://www.sqlservercentral.com/articles/solve-sudoku-with-t-sql-part-1-1

In [ ]:
select version();

/* 81 long number table of row, column and block mapping */

In [ ]:
create table tGrid (CoordinateID int primary key, RowNum int, ColumnNum int, BlockNum int);

In [ ]:
INSERT INTO tGrid
SELECT seq+1 AS CoordinateID,
       seq div 9 +1 AS RowNum,
       seq MOD 9 +1 AS ColumnNum,
               (seq div 27)*3 + 1 + ((seq div 3) MOD 3) AS BlockNum
FROM seq_0_to_80;

In [ ]:
select * from tGrid order by CoordinateID limit 28;

Table of what we've already solved

In [ ]:
CREATE OR REPLACE TABLE tFixed (CoordinateID INT PRIMARY KEY, Value CHAR(1));

In [ ]:
set @PuzzleIn = '023780460000620000060304080001000534280000097439000100010205040000036000056018370';

In [ ]:
INSERT INTO tFixed (CoordinateID, Value)
SELECT CoordinateID, SUBSTRING(@PuzzleIn, CoordinateID, 1) AS Value
FROM tGrid
WHERE SUBSTRING(@PuzzleIn, CoordinateID, 1) != '0';

In [ ]:
CREATE OR REPLACE TABLE tOption (CoordinateID INT, Value INT, Key (CoordinateID));

All possible values

In [ ]:
SELECT CoordinateID, b.seq FROM tGrid AS a CROSS JOIN seq_1_to_9 AS b;

All possible values, excluding known values that are fixed

In [ ]:
SELECT CoordinateID,
       b.seq
FROM tGrid AS a
LEFT JOIN tFixed f USING (CoordinateID)
CROSS JOIN seq_1_to_9 AS b
WHERE f.CoordinateID IS NULL;

Same query using subquery

In [ ]:
SELECT CoordinateID,
       b.seq
FROM tGrid AS a
CROSS JOIN seq_1_to_9 AS b
WHERE NOT EXISTS
    (SELECT 1
     FROM tFixed c
     WHERE a.CoordinateID = c.CoordinateID );

In addition to exluding the fixed values we know about, exclude same value on the same row

AND NOT EXISTS (
  SELECT 1 FROM tFixed c
  INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
  WHERE a.RowNum = d.RowNum AND b.seq = c.Value
)

And same value on the same column

AND NOT EXISTS (
  SELECT 1 FROM tFixed c
  INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
  WHERE a.ColumnNum = d.ColumnNum AND b.seq = c.Value
)

And same value in the same grid

AND NOT EXISTS (
  SELECT 1 FROM tFixed c
  INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
  WHERE a.BlockNum = d.BlockNum AND b.seq = c.Value
)

And to this list of options append the fixed values we had

UNION
SELECT CoordinateID, Value FROM tFixed;

Resulting query

In [ ]:
INSERT INTO tOption (CoordinateID, Value)
SELECT CoordinateID,
       seq
FROM tGrid AS a
CROSS JOIN seq_1_to_9 AS b
WHERE NOT EXISTS
    (SELECT 1
     FROM tFixed c
     WHERE a.CoordinateID = c.CoordinateID )
  AND NOT EXISTS
    (SELECT 1
     FROM tFixed c
     INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
     WHERE a.RowNum = d.RowNum
       AND b.seq = c.Value )
  AND NOT EXISTS
    (SELECT 1
     FROM tFixed c
     INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
     WHERE a.ColumnNum = d.ColumnNum
       AND b.seq = c.Value )
  AND NOT EXISTS
    (SELECT 1
     FROM tFixed c
     INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
     WHERE a.BlockNum = d.BlockNum
       AND b.seq = c.Value )
UNION
SELECT CoordinateID,
       Value
FROM tFixed;

Duplicates exist

In [ ]:
select * from tOption order by CoordinateID limit 10;

So with 81 Coordinates (9 x 9 grid), cross join them all. Use SQL to generate SQL

In [ ]:
SELECT @SelectQuery := GROUP_CONCAT(' (SELECT Value AS `',
                                    CoordinateID,
                                    '` FROM tOption WHERE CoordinateID = ',
                                    CoordinateID,
                                    ') AS `',
                                    CoordinateID,
                                    '`'
                                    SEPARATOR ' CROSS JOIN ')
FROM
  (SELECT DISTINCT CoordinateID
   FROM tOption) AS b;

In [ ]:
SELECT @ExclusionQuery := GROUP_CONCAT('`', l.CoordinateID, '` <> `', r.CoordinateID, '`' SEPARATOR ' AND ')
FROM tGrid l
CROSS JOIN tGrid r
WHERE (l.RowNum = r.RowNum
       OR l.ColumnNum = r.ColumnNum
       OR l.BlockNum = r.BlockNum)
  AND r.CoordinateID > l.CoordinateID;

In [ ]:
select @columns := group_concat('`', seq, '`' SEPARATOR ',') from seq_1_to_81;

In [ ]:
EXECUTE IMMEDIATE concat('SELECT ROW_NUMBER() OVER (ORDER BY `1` ASC) AS SolutionID, ',
                         @columns,
                         ' FROM ( SELECT * FROM',
                         @SelectQuery,
                         ' WHERE ',
                         @ExclusionQuery,
                        ') AS x');

In [ ]:
SELECT @SelectQuery50 := GROUP_CONCAT(' (SELECT Value AS `', CoordinateID, '` FROM tOption WHERE CoordinateID = ', CoordinateID, ') AS `', CoordinateID, '`' SEPARATOR ' CROSS JOIN ')
FROM
  (SELECT seq AS CoordinateID
   FROM seq_1_to_50) b;

In [ ]:
SELECT @ExclusionQuery50 := GROUP_CONCAT('`', l.CoordinateID, '` <> `', r.CoordinateID, '`' SEPARATOR ' AND ')
FROM tGrid l
CROSS JOIN tGrid r
WHERE (l.RowNum = r.RowNum
       OR l.ColumnNum = r.ColumnNum
       OR l.BlockNum = r.BlockNum)
  AND r.CoordinateID > l.CoordinateID
  AND r.CoordinateID <= 50
  AND l.CoordinateID <= 50;

In [ ]:
EXECUTE IMMEDIATE concat('create or replace table tOption50( CoordinateID int, key (CoordinateID))  select * FROM ',
                         @SelectQuery50,
                        ' WHERE ',
                        @ExclusionQuery50);

In [ ]:
SELECT @SelectQuery81 := GROUP_CONCAT(' (SELECT Value AS `', CoordinateID, '` FROM tOption WHERE CoordinateID = ', CoordinateID, ') AS `', CoordinateID, '`' SEPARATOR ' CROSS JOIN ')
FROM
  (SELECT seq AS CoordinateID
   FROM seq_51_to_81) b;

In [ ]:
EXECUTE IMMEDIATE concat('SELECT ROW_NUMBER() OVER (ORDER BY `1` ASC) AS SolutionID, ',
                         @columns,
                        ' FROM ( SELECT * FROM tOption50 CROSS JOIN ',
                        @SelectQuery81,
                        ' WHERE ',
                        @ExclusionQuery,
                        ') AS x');

In [ ]:
set @PuzzleIn = '023780460000620000060304080001000534280000097439000100010205040000036000056018370';







In [ ]:
set @PuzzleIn = '600050002010702040000346000084000590509000207032000480000165000020407060300090004';

In [ ]:
set @PuzzleIn = '800259004040010070000407000302080506580302091607040302000504000060090020700826009';


In [ ]:
set @PuzzleIn = '703200104054019380000500000070000805060000030308000090000001000035920460407008903';

In [ ]:
set @PuzzleIn = '700200008020405070004080300060508037009000800180607020001070600050903010600004003';

In [ ]:
set @PuzzleIn = '000400000004063200809000503090030005040658030600010070901000804005840100000007000';

In [ ]:
set @PuzzleIn = '007400803000206001000085700026000039004000100370000680908620000000108000630009200';


In [ ]:
set @PuzzleIn = '019600430000098000002005100098074001020000080700850620007500800000730000056001970';


In [ ]:
set @PuzzleIn = '280060079100007006070930080907000605008000700040000090090025060800600002650010047';

In [ ]:
set @PuzzleIn = '107608209840050036000000000300106004060000090200305007000000000430060052508209301';